In [17]:
from dputils import scrape as sc
from bs4 import BeautifulSoup

In [18]:
soup = sc.get_webpage_data('https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/dp/B0812JV4L6/ref=sr_1_1_sspa?crid=1MNRSZBINEA3A&keywords=ruler&qid=1661832448&sprefix=ruler%2Caps%2C129&sr=8-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExNTJWQ1VIMEZFTEUmZW5jcnlwdGVkSWQ9QTA4OTIxOTAxV1kyWUlZWk85SFIyJmVuY3J5cHRlZEFkSWQ9QTAxODM3OTFIWkxRNjBTNVVQWE0md2lkZ2V0TmFtZT1zcF9hdGYmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl&th=1')

In [19]:
review_link_area = soup.find('div',attrs={'id':'reviews-medley-footer'})
all_review_link = review_link_area.find('a').get('href')
print(all_review_link)

/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [20]:
final_url = 'https://www.amazon.com' + all_review_link
print(final_url)

https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [21]:
def get_review_link(product_url="", base_url='https://www.amazon.com'):
    try:
        soup = sc.get_webpage_data(product_url)
        review_link_area = sc.extract_one(soup, div={'tag':'div','attrs':{'id':'reviews-medley-footer'},'output':'object'})
        all_review_link = review_link_area['div'].find('a').get('href')
        return base_url + all_review_link
    except Exception as e:
        print(e)
        return None


In [22]:
target = {'tag':'div','attrs':{'id':'cm_cr-review_list'}} # the page subpart
items= {'tag':'div','attrs':{'class':'a-section review aok-relative'}} # the repeating elements
reviewer = {'tag':'span','attrs':{'class':'a-profile-name'}} # the name inside a single repeating element
review = {'tag':'span','attrs':{'class':'a-size-base review-text review-text-content'}} # the review inside a single repeating element

In [23]:
page = 1
product_url = 'https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/dp/B0812JV4L6/ref=sr_1_1_sspa?crid=1MNRSZBINEA3A&keywords=ruler&qid=1661832448&sprefix=ruler%2Caps%2C129&sr=8-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExNTJWQ1VIMEZFTEUmZW5jcnlwdGVkSWQ9QTA4OTIxOTAxV1kyWUlZWk85SFIyJmVuY3J5cHRlZEFkSWQ9QTAxODM3OTFIWkxRNjBTNVVQWE0md2lkZ2V0TmFtZT1zcF9hdGYmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl&th=1'
all_reviews_link = get_review_link(product_url)
datalist = []
limit = 0
while True:
    review_link = all_reviews_link + '&pageNumber=' + str(page)
    print('=>>>',review_link)
    review_soup = sc.get_webpage_data(review_link)
    output =sc.extract_many(review_soup, target=target, items=items, reviewer=reviewer, content= review)
    if output is None:
        print("LOG: No more reviews, output is None")
        break
    elif len(output) == 0:
        print("LOG: No more reviews, output is empty")
        break
    elif limit > 100:
        print(f"LOG: No more reviews, limit is {limit}")
        break
    else:
        datalist.extend(output)
        page += 1
        limit += 1
print(len(datalist))

=>>> https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
10 items found
All items extracted
=>>> https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
10 items found
All items extracted
=>>> https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
10 items found
All items extracted
=>>> https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
10 items found
All items extracted
=>>> https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5


In [ ]:
len(datalist)

1010

In [45]:
import pandas as pd

In [46]:
pd.DataFrame(datalist)

,reviewer,content
0,K. MURPHY,"UPDATE 12-15-2020 - After three weeks, I'm sti..."
1,Try it or DIY it,Beautiful laptop. I’ve upgraded from the MacBo...
2,Impetigo,"I have two MacBook Airs (MBA), a 13"" and a 11,..."
3,Paul K.,Liked:- Long battery life (one charge WILL las...
4,Adi,I'm a fan of apple products and I like almost ...
...,...,...
1005,lynellw,Great purchase
1006,arlene r.,My first Apple product. So far finding it diff...
1007,Gautam,Product was broken
1008,Jus me,My child is taking class on this Laptop


In [15]:
soup = sc.get_webpage_data('https://www.amazon.com/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/dp/B0812JV4L6/ref=sr_1_1_sspa?crid=1MNRSZBINEA3A&keywords=ruler&qid=1661832448&sprefix=ruler%2Caps%2C129&sr=8-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExNTJWQ1VIMEZFTEUmZW5jcnlwdGVkSWQ9QTA4OTIxOTAxV1kyWUlZWk85SFIyJmVuY3J5cHRlZEFkSWQ9QTAxODM3OTFIWkxRNjBTNVVQWE0md2lkZ2V0TmFtZT1zcF9hdGYmYWN0aW9uPWNsaWNrUmVkaXJlY3QmZG9Ob3RMb2dDbGljaz10cnVl&th=1')
review_link_area = sc.extract_one(soup, div={'tag':'div','attrs':{'id':'reviews-medley-footer'},'output':'object'})

In [16]:
review_link_area

{'div': <div class="a-section" data-hook="reviews-medley-footer" id="reviews-medley-footer"><div class="a-row"></div><div class="a-row a-spacing-medium"><hr aria-hidden="true" class="a-spacing-base a-divider-normal"/><a class="a-link-emphasis a-text-bold" data-hook="see-all-reviews-link-foot" href="/AmazonBasics-6-Inch-Stainless-Steel-4-Pack/product-reviews/B0812JV4L6/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&amp;reviewerType=all_reviews">See all reviews</a></div></div>}